In [25]:
import unicodedata

import re


# Convert the unicode sequence to ascii
def unicode_to_ascii(s):

  # Normalize the unicode string and remove the non-spacking mark
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')

# Preprocess the sequence
def preprocess_sentence(w):

  # Clean the sequence
  w = unicode_to_ascii(w.lower().strip())
  
  # Create a space between a word and the punctuation following it also place a space between the punctuation and the following word. Note that punctuation also includes | 

  w = re.sub(r"([?.!।])", r" \1 ", w)
  
  # Add a start and stop token to detect the start and end of the sequence
  w = '<start> ' + w + ' <end>'
  return w

In [26]:
import io

# Create the Dataset
def create_dataset(path):
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

  # Loop through lines (sequences) and extract the English and French sequences. Store them as a word-pair
  word_pairs = [[preprocess_sentence(w) for w in l.split('\t', 2)[:-1]]  for l in lines]
  return zip(*word_pairs)

In [27]:
path_to_file='ben.txt'

In [28]:
lines = io.open(path_to_file, encoding='UTF-8').read().strip().split('\n')
print(lines[0])
print(preprocess_sentence(lines[0].split('\t', 2)[0]))
print(preprocess_sentence(lines[0].split('\t', 2)[1]))

Go.	যাও।	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #5545004 (tanay)
<start> go .  <end>
<start> যাও ।  <end>


In [29]:
en, fra = create_dataset(path_to_file)
print(en[0])
print(fra[0])

<start> go .  <end>
<start> যাও ।  <end>


In [30]:
import tensorflow as tf

# Convert sequences to tokenizers
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  
  # Convert sequences into internal vocab
  lang_tokenizer.fit_on_texts(lang)

  # Convert internal vocab to numbers
  tensor = lang_tokenizer.texts_to_sequences(lang)

  # Pad the tensors to assign equal length to all the sequences
  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

In [31]:
# Load the dataset
def load_dataset(path, num_examples=None):
 
  # Create dataset (targ_lan = English, inp_lang = French)
  inp_lang,targ_lang = create_dataset(path)

  # Tokenize the sequences
  input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [32]:
# Consider 50k examples
num_examples = 50000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

In [33]:
print(max_length_targ, max_length_inp)

21 23


In [34]:
print(input_tensor[0], target_tensor.shape)

[ 1 33  3  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0] (6509, 21)


In [35]:
from sklearn.model_selection import train_test_split

# Create training and validation sets using an 80/20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

5207 5207 1302 1302


In [36]:
print(input_tensor_train)
print(inp_lang)

[[  1 237  27 ...   0   0   0]
 [  1  22  10 ...   0   0   0]
 [  1 119  20 ...   0   0   0]
 ...
 [  1   4  68 ...   0   0   0]
 [  1  23 388 ...   0   0   0]
 [  1   7 817 ...   0   0   0]]


In [37]:
# Show the mapping b/w word index and language tokenizer
def convert(lang, tensor):
  for t in tensor:
    if t != 0:
      print ("%d ----> %s" % (t, lang.index_word[t]))
      
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
1 ----> <start>
237 ----> who's
27 ----> your
1214 ----> aunt
5 ----> ?
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
49 ----> কে
14 ----> আপনার
1605 ----> কাকি
5 ----> ?
2 ----> <end>


In [38]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index) + 1
vocab_tar_size = len(targ_lang.word_index) + 1

In [39]:
dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [40]:
# Size of input and target batches
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 23]), TensorShape([64, 21]))

In [41]:
import tensorflow as tf

class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units

        # Embedding layer
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)

        # LSTM Layer
        self.lstm = tf.keras.layers.LSTM(self.enc_units,
                                         return_sequences=True,
                                         return_state=True,
                                         recurrent_initializer='glorot_uniform')

    # Encoder network comprises an Embedding layer followed by an LSTM layer
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state_h, state_c = self.lstm(x, initial_state=hidden)
        state = [state_h, state_c]
        return output, state

    # To initialize the hidden state
    def initialize_hidden_state(self):
        return [tf.zeros((self.batch_sz, self.enc_units)), tf.zeros((self.batch_sz, self.enc_units))]


In [42]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)

print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden[0].shape))
print ('Encoder Cell state shape: (batch size, units) {}'.format(sample_hidden[1].shape))


Encoder output shape: (batch size, sequence length, units) (64, 23, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)
Encoder Cell state shape: (batch size, units) (64, 1024)


In [43]:
import tensorflow as tf

class PayAttention(tf.keras.layers.Layer):
    def __init__(self, units, length):
        self.units=units
        self.length=length
        super(PayAttention, self).__init__() #Call initializer of the superclass

    def build(self, input_shape):
        self.w = self.add_weight(shape=(self.units,1), initializer='normal')
        self.b = self.add_weight(shape=(self.length,1), initializer='zeros')
        super(PayAttention, self).build(input_shape)

    def call(self, sentences):
        # Sentences = (batch_size, length of sentence, units)
        E = tf.nn.tanh(tf.keras.backend.dot(sentences,self.w)+self.b)
        # E = (batch_size, length, 1)
        A = tf.nn.softmax(E, axis=1)
        # A = softmax(E) = (batch_size, length, 1)
        out= A*sentences
        # out = (batch_size, length, units)
        return tf.keras.backend.sum(out, axis=1), A # (batch_size, units), (batch_size, length, 1)
 
    


In [44]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz,inp_units, enc_length):
        super().__init__()
        self.batch_sz=batch_sz
        self.dec_units=dec_units
        self.embedding_dim=embedding_dim
        self.vocab_size=vocab_size 

        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim) 

        self.attention = PayAttention(inp_units, enc_length)
        
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

        self.fc = tf.keras.layers.Dense(vocab_size)        

    def call(self, enc_output, dec_input):
        # hidden is the hidden states of all the units in the encoder
        # context_vector is the context vector from the attention layer
        # dec_input is the input to the decoder

        # Now, first embed the decoder input
        x=self.embedding(dec_input)
        
        context_vector, attention_weights = self.attention(enc_output)
        
        # Now we will concat the encoder input and the context vector
            #first expand the context vector
        context_vector=tf.expand_dims(context_vector,1)
        x = tf.concat([context_vector, x], axis=-1)

        # Pass through a GRU layer
        output,state = self.gru(x)

        # Pass through a dense layer to get the probabilities distribution over the target vocabulary
        output = tf.reshape(output, (-1, output.shape[2]))
        x = self.fc(output)

        return x, state , attention_weights

In [45]:
#Now we define the loss function. The loss function is the cross entropy loss function. The cross entropy loss function is defined as follows:
#-sum(y_true * log(y_pred), axis=-1)
import numpy as np

optimizer = tf.optimizers.Adam()

def loss_function(real, pred):
    mask = 1 - np.equal(real, 0)
    loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
    return tf.reduce_mean(loss_)

In [46]:

#Now we run the training loop

optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

decoder=DecoderLayer(vocab_tar_size, embedding_dim, units, BATCH_SIZE, units, max_length_inp)
encoder=Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

import os
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

def train_step(inp, targ):
    loss=0

    with tf.GradientTape() as tape:
        
        # Initialize the hidden state of the encoder and pass the input to the encoder
        hidden_initialize = encoder.initialize_hidden_state()
        enc_output, enc_hidden = encoder(inp, hidden_initialize)

        # Initialize the hidden state of the decoder
        dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)   

        # Teacher forcing - feeding the target as the next input
        for t in range(1, targ.shape[1]):
            # passing enc_output to the decoder
            predictions, dec_hidden, _ = decoder(enc_output, dec_input)
            loss += loss_function(targ[:, t], predictions)
            # using teacher forcing
            dec_input = tf.expand_dims(targ[:, t], 1)
        

    batch_loss = (loss / int(targ.shape[1]))
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    return batch_loss


    

In [47]:
print(steps_per_epoch)

81


In [48]:
EPOCHS=20
import time

for epoch in range(EPOCHS):
    start = time.time()
    total_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ)
        total_loss += batch_loss
        print(targ.shape)
        print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.numpy()))
    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
        checkpoint.save(file_prefix = checkpoint_prefix+str(epoch))

    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))



(64, 21)
Epoch 1 Batch 0 Loss 2.6109
(64, 21)
Epoch 1 Batch 1 Loss 2.5136
(64, 21)
Epoch 1 Batch 2 Loss 2.5645
(64, 21)
Epoch 1 Batch 3 Loss 2.2918
(64, 21)
Epoch 1 Batch 4 Loss 2.1355
(64, 21)
Epoch 1 Batch 5 Loss 1.8477
(64, 21)
Epoch 1 Batch 6 Loss 1.9221
(64, 21)
Epoch 1 Batch 7 Loss 1.8126
(64, 21)
Epoch 1 Batch 8 Loss 1.6980
(64, 21)
Epoch 1 Batch 9 Loss 1.8674
(64, 21)
Epoch 1 Batch 10 Loss 2.1525
(64, 21)
Epoch 1 Batch 11 Loss 1.8862
(64, 21)
Epoch 1 Batch 12 Loss 1.8881
(64, 21)
Epoch 1 Batch 13 Loss 1.9709
(64, 21)
Epoch 1 Batch 14 Loss 1.8169
(64, 21)
Epoch 1 Batch 15 Loss 1.7177
(64, 21)
Epoch 1 Batch 16 Loss 1.6843
(64, 21)
Epoch 1 Batch 17 Loss 1.8598
(64, 21)
Epoch 1 Batch 18 Loss 1.9808
(64, 21)
Epoch 1 Batch 19 Loss 1.7288
(64, 21)
Epoch 1 Batch 20 Loss 1.6829
(64, 21)
Epoch 1 Batch 21 Loss 1.7677
(64, 21)
Epoch 1 Batch 22 Loss 1.6307
(64, 21)
Epoch 1 Batch 23 Loss 1.8804
(64, 21)
Epoch 1 Batch 24 Loss 1.6944
(64, 21)
Epoch 1 Batch 25 Loss 1.6233
(64, 21)
Epoch 1 Batch

2024-05-16 15:30:31.692620: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


(64, 21)
Epoch 2 Batch 0 Loss 1.4677
(64, 21)
Epoch 2 Batch 1 Loss 1.4001
(64, 21)
Epoch 2 Batch 2 Loss 1.4374
(64, 21)
Epoch 2 Batch 3 Loss 1.5652
(64, 21)
Epoch 2 Batch 4 Loss 1.2603
(64, 21)
Epoch 2 Batch 5 Loss 1.5318
(64, 21)
Epoch 2 Batch 6 Loss 1.4232
(64, 21)
Epoch 2 Batch 7 Loss 1.3554
(64, 21)
Epoch 2 Batch 8 Loss 1.4330
(64, 21)
Epoch 2 Batch 9 Loss 1.4405
(64, 21)
Epoch 2 Batch 10 Loss 1.4820
(64, 21)
Epoch 2 Batch 11 Loss 1.5569
(64, 21)
Epoch 2 Batch 12 Loss 1.4050
(64, 21)
Epoch 2 Batch 13 Loss 1.4535
(64, 21)
Epoch 2 Batch 14 Loss 1.6454
(64, 21)
Epoch 2 Batch 15 Loss 1.5117
(64, 21)
Epoch 2 Batch 16 Loss 1.3190
(64, 21)
Epoch 2 Batch 17 Loss 1.5107
(64, 21)
Epoch 2 Batch 18 Loss 1.4179
(64, 21)
Epoch 2 Batch 19 Loss 1.4391
(64, 21)
Epoch 2 Batch 20 Loss 1.5568
(64, 21)
Epoch 2 Batch 21 Loss 1.4665
(64, 21)
Epoch 2 Batch 22 Loss 1.4623
(64, 21)
Epoch 2 Batch 23 Loss 1.4741
(64, 21)
Epoch 2 Batch 24 Loss 1.5336
(64, 21)
Epoch 2 Batch 25 Loss 1.6366
(64, 21)
Epoch 2 Batch

2024-05-16 15:34:36.718338: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 2 Loss 1.4497
Time taken for 1 epoch 245.64204716682434 sec

(64, 21)
Epoch 3 Batch 0 Loss 1.3370
(64, 21)
Epoch 3 Batch 1 Loss 1.4808
(64, 21)
Epoch 3 Batch 2 Loss 1.2753
(64, 21)
Epoch 3 Batch 3 Loss 1.3322
(64, 21)
Epoch 3 Batch 4 Loss 1.2838
(64, 21)
Epoch 3 Batch 5 Loss 1.2901
(64, 21)
Epoch 3 Batch 6 Loss 1.2254
(64, 21)
Epoch 3 Batch 7 Loss 1.3818
(64, 21)
Epoch 3 Batch 8 Loss 1.3775
(64, 21)
Epoch 3 Batch 9 Loss 1.3373
(64, 21)
Epoch 3 Batch 10 Loss 1.3740
(64, 21)
Epoch 3 Batch 11 Loss 1.3700
(64, 21)
Epoch 3 Batch 12 Loss 1.2589
(64, 21)
Epoch 3 Batch 13 Loss 1.3935
(64, 21)
Epoch 3 Batch 14 Loss 1.2512
(64, 21)
Epoch 3 Batch 15 Loss 1.3989
(64, 21)
Epoch 3 Batch 16 Loss 1.2429
(64, 21)
Epoch 3 Batch 17 Loss 1.4185
(64, 21)
Epoch 3 Batch 18 Loss 1.2523
(64, 21)
Epoch 3 Batch 19 Loss 1.3816
(64, 21)
Epoch 3 Batch 20 Loss 1.4095
(64, 21)
Epoch 3 Batch 21 Loss 1.2782
(64, 21)
Epoch 3 Batch 22 Loss 1.2474
(64, 21)
Epoch 3 Batch 23 Loss 1.3091
(64, 21)
Epoch 3 Batch 24 Loss 

...

Epoch 18 Loss 0.0981
Time taken for 1 epoch 257.386803150177 sec

(64, 21)
Epoch 19 Batch 0 Loss 0.0740
(64, 21)
Epoch 19 Batch 1 Loss 0.0654
(64, 21)
Epoch 19 Batch 2 Loss 0.0900
(64, 21)
Epoch 19 Batch 3 Loss 0.0838
(64, 21)
Epoch 19 Batch 4 Loss 0.0916
(64, 21)
Epoch 19 Batch 5 Loss 0.0408
(64, 21)
Epoch 19 Batch 6 Loss 0.0873
(64, 21)
Epoch 19 Batch 7 Loss 0.0522
(64, 21)
Epoch 19 Batch 8 Loss 0.0815
(64, 21)
Epoch 19 Batch 9 Loss 0.0944
(64, 21)
Epoch 19 Batch 10 Loss 0.0887
(64, 21)
Epoch 19 Batch 11 Loss 0.1147
(64, 21)
Epoch 19 Batch 12 Loss 0.0841
(64, 21)
Epoch 19 Batch 13 Loss 0.0589
(64, 21)
Epoch 19 Batch 14 Loss 0.0986
(64, 21)
Epoch 19 Batch 15 Loss 0.0802
(64, 21)
Epoch 19 Batch 16 Loss 0.0908
(64, 21)
Epoch 19 Batch 17 Loss 0.0715
(64, 21)
Epoch 19 Batch 18 Loss 0.0883
(64, 21)
Epoch 19 Batch 19 Loss 0.0604
(64, 21)
Epoch 19 Batch 20 Loss 0.0746
(64, 21)
Epoch 19 Batch 21 Loss 0.0947
(64, 21)
Epoch 19 Batch 22 Loss 0.0680
(64, 21)
Epoch 19 Batch 23 Loss 0.0807
(64, 21)

2024-05-16 16:45:21.228318: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


(64, 21)
Epoch 20 Batch 0 Loss 0.0477
(64, 21)
Epoch 20 Batch 1 Loss 0.0525
(64, 21)
Epoch 20 Batch 2 Loss 0.0586
(64, 21)
Epoch 20 Batch 3 Loss 0.0577
(64, 21)
Epoch 20 Batch 4 Loss 0.0948
(64, 21)
Epoch 20 Batch 5 Loss 0.0799
(64, 21)
Epoch 20 Batch 6 Loss 0.0617
(64, 21)
Epoch 20 Batch 7 Loss 0.0527
(64, 21)
Epoch 20 Batch 8 Loss 0.0827
(64, 21)
Epoch 20 Batch 9 Loss 0.0781
(64, 21)
Epoch 20 Batch 10 Loss 0.0589
(64, 21)
Epoch 20 Batch 11 Loss 0.0978
(64, 21)
Epoch 20 Batch 12 Loss 0.0483
(64, 21)
Epoch 20 Batch 13 Loss 0.0684
(64, 21)
Epoch 20 Batch 14 Loss 0.0900
(64, 21)
Epoch 20 Batch 15 Loss 0.0760
(64, 21)
Epoch 20 Batch 16 Loss 0.0903
(64, 21)
Epoch 20 Batch 17 Loss 0.0760
(64, 21)
Epoch 20 Batch 18 Loss 0.0811
(64, 21)
Epoch 20 Batch 19 Loss 0.0745
(64, 21)
Epoch 20 Batch 20 Loss 0.0706
(64, 21)
Epoch 20 Batch 21 Loss 0.0753
(64, 21)
Epoch 20 Batch 22 Loss 0.0729
(64, 21)
Epoch 20 Batch 23 Loss 0.0947
(64, 21)
Epoch 20 Batch 24 Loss 0.0853
(64, 21)
Epoch 20 Batch 25 Loss 0.0

2024-05-16 16:49:23.290364: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 20 Loss 0.0797
Time taken for 1 epoch 242.66775918006897 sec



In [49]:
checkpoint.save(file_prefix = checkpoint_prefix)

'./training_checkpoints/ckpt-11'

In [103]:
max_seq_length = 20
#Now we will generate the translation of the input sentence

def evaluate(sentence):
    
    sentence = preprocess_sentence(sentence)
    print(sentence)
    sentence = inp_lang.texts_to_sequences([sentence])
    print(sentence)
    len_sent=len(sentence[0])
    # Pad the sentence to the max_length_inp

    sentence = tf.keras.preprocessing.sequence.pad_sequences(sentence,
                                                            maxlen=max_length_inp,
                                                            padding='post')

    hidden_initialize = encoder.initialize_hidden_state()
    enc_output, enc_hidden = encoder(sentence, hidden_initialize)
    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    result = []
    #Disable teacher forcing
    attention_graph=[]
    for i in range(max_seq_length):
        predictions, dec_hidden, attention_weights = decoder(enc_output, dec_input)
        predicted_id = tf.argmax(predictions[0]).numpy()
        attention_graph.append(attention_weights)
        result.append(targ_lang.index_word[predicted_id])

        if targ_lang.index_word[predicted_id] == '<end>':
            returnd=(attention_graph,result,len_sent)
            return returnd

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id] * BATCH_SIZE, 0)
        dec_input = tf.transpose(dec_input)
        print(dec_input.shape)
    
    returnd=(attention_graph,result,len_sent)
    return returnd


resultas = evaluate("Let's go home.")

attend, result, len_inp = resultas[0], resultas[1], resultas[2]


<start> let's go home .  <end>
[[1, 179, 33, 135, 3, 2]]
(64, 1)
(64, 1)
(64, 1)
(64, 1)


In [104]:
print(len(result))

5


In [105]:
print(attend[0].shape)

(64, 23, 1)


In [106]:
new_attend=[]
for i in range(len(attend)):
    new_attend.append(attend[i][0][:len_inp])

In [107]:
print(len_inp)

6


In [108]:
print(len(new_attend))

5


In [109]:
print(new_attend[0].shape)

(6, 1)


In [110]:
new_attend=np.array(new_attend)

In [112]:
print(new_attend)

[[[0.02573219]
  [0.02547985]
  [0.02335659]
  [0.02211129]
  [0.0225455 ]
  [0.0268532 ]]

 [[0.02573219]
  [0.02547985]
  [0.02335659]
  [0.02211129]
  [0.0225455 ]
  [0.0268532 ]]

 [[0.02573219]
  [0.02547985]
  [0.02335659]
  [0.02211129]
  [0.0225455 ]
  [0.0268532 ]]

 [[0.02573219]
  [0.02547985]
  [0.02335659]
  [0.02211129]
  [0.0225455 ]
  [0.0268532 ]]

 [[0.02573219]
  [0.02547985]
  [0.02335659]
  [0.02211129]
  [0.0225455 ]
  [0.0268532 ]]]
